In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
dataset_path = 'titanic_modified_dataset.csv'
df = pd.read_csv(dataset_path, index_col='PassengerId')

In [5]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Survived
PassengerId,,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0,0
2,1,1,38.0,1,0,71.2833,1,1,1
3,3,1,26.0,0,0,7.9250,0,2,1
4,1,1,35.0,1,0,53.1000,0,1,1
5,3,0,35.0,0,0,8.0500,0,0,0
...,...,...,...,...,...,...,...,...,...
887,2,0,27.0,0,0,13.0000,0,5,0
888,1,1,19.0,0,0,30.0000,0,2,1
889,3,1,28.0,1,2,23.4500,0,2,0


In [ ]:
dataset_arr = df.to_numpy().astype(np.float64)
X, y = dataset_arr[:, :-1], dataset_arr[:, -1]

print(X)
print(y)

In [9]:
intercept = np.ones((X.shape[0], 1))
X_b = np.concatenate((intercept, X), axis=1)

print(X_b)

[[ 1.      3.      0.     ...  7.25    0.      0.    ]
 [ 1.      1.      1.     ... 71.2833  1.      1.    ]
 [ 1.      3.      1.     ...  7.925   0.      2.    ]
 ...
 [ 1.      3.      1.     ... 23.45    0.      2.    ]
 [ 1.      1.      0.     ... 30.      1.      0.    ]
 [ 1.      3.      0.     ...  7.75    2.      0.    ]]


In [13]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X_b, y, 
    test_size=val_size,
    random_state=random_state,
    shuffle=is_shuffle,
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, 
    test_size=test_size,
    random_state=random_state,
    shuffle=is_shuffle
)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(623, 9)
(89, 9)
(179, 9)


In [ ]:
normalize = StandardScaler()
X_train[:, 1:] = normalize.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalize.fit_transform(X_val[:, 1:])
X_test[:, 1:] = normalize.fit_transform(X_test[:, 1:])

print(X_train[0])

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def predict(X, theta):
    dot_product = np.dot(X, theta)
    y_hat = sigmoid(dot_product)
    return y_hat

def compute_loss(y_hat, y):
    y_hat = np.clip(y_hat, 1e-7, 1 - 1e-7)
    
    return (-y * np.log(y_hat) - (1-y)*np.log(1-y_hat)).mean()

def compute_gradient(X, y, y_hat):
    return np.dot(X.T, (y_hat-y))/y.size

def update_theta(theta, gradient, lr):
    return theta - lr*gradient

def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta).round()
    acc = (y_hat == y).mean()
    
    return acc

In [4]:
X = [[22.3, -1.5, 1.1, 1]]
theta =  [0.1,-0.15, 0.3,-0.2]
print(predict(X, theta))

[0.92988994]


In [5]:
y = np.array([1, 0, 0, 1])
y_hat = np.array([0.8, 0.75, 0.3,0.95])

print(compute_loss(y_hat, y))

0.5043515376900958


In [3]:
def compute_gradient(X, y_true, y_pred):
    gradient = np.dot(X.T, (y_pred-y_true)) / y_true.size
    return gradient

X = [[1, 2], [2, 1], [1, 1], [2, 2]]
y_true = [0, 1, 0, 1]
y_pred = [0.25, 0.75,0.4, 0.8]

X = np.array(X)
y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(compute_gradient(X, y_true, y_pred))

[-0.0625  0.0625]


In [6]:
X = [[1, 3], [2, 1], [3, 2], [1, 2]]
y_true = [1, 0, 1, 1]
y_pred = [0.7, 0.4, 0.6, 0.85]

X = np.array(X)
y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(compute_gradient(X, y_true, y_pred))

[-0.2125 -0.4   ]


In [9]:
def compute_accuracy(y_true, y_pred):
    y_pred_rounded = np.round(y_pred)
    accuracy = np.mean(y_true == y_pred_rounded)
    return accuracy

y_true = [1, 0, 1, 1] 
y_pred = [0.85, 0.35, 0.9, 0.75] 

print(compute_accuracy(y_true, y_pred))

1.0


In [16]:
lr = 0.01
epochs = 100
batch_size = 16

np.random.seed(random_state)
theta = np.random.uniform(size=X_train.shape[1])